In [3]:
#-*- coding:utf-8 -*-
import requests        # 手机抓包爬虫一般没什么反爬机制，requests 就够用
import pandas as pd    # 爬下来的东西存成 DataFrame
import numpy as np     # 爬下来的薪水是一个区间，用 np 对数组进行一个运算
import pymysql         # 测试数据库的连接，在 jupyter 上用 sql 对数据库进行增删改查
from sqlalchemy import create_engine # 一个可以将 DataFrame 直接传到数据库的 engine

In [5]:
def sxs_crawl(pages, kw, c):
    list_urls = ["https://iosapi.shixiseng.com/app/interns/search?c={}&d=&ft=&i=&k={}"
                "&m=&page={}&s=-0&st=&t=zj&x=&z=".format(c, kw, page) for page in range(pages)]
    job_list_data = []
    for url in list_urls:
        response = requests.get(url)
        if response.json()['msg']:
            job_list_data.extend(response.json()['msg'])
        else:
            break
    job_list = pd.DataFrame(job_list_data)
    job_list.to_csv(r'./job_list.csv', index=False)
    print('Successfully crawled {} jobs list')
    
    # 职位详情ID爬取
    uuids = list(job_list['uuid'])
    job_detailed_url = ['https://iosapi.shixiseng.com/app/intern/info?uuid={}'.format(uuid) for uuid in uuids]

    job_detailed_data = []
    for url in job_detailed_url:
        response = requests.get(url)
        job_detailed_data.append(response.json()['msg'])
    job_detailed = pd.DataFrame(job_detailed_data)
    job_detailed.to_csv(r'./job_detailed.csv', index=False)

    # 公司信息爬取
    cuuids = list(job_detailed['cuuid'])
    com_detailed_url = ['https://iosapi.shixiseng.com/app/company/info?uuid={}'.format(cuuid) for cuuid in cuuids]
    com_detailed_data = []
    for url in com_detailed_url:
        response = requests.get(url)
        com_detailed_data.append(response.json()['msg'])
    com_detailed = pd.DataFrame(com_detailed_data)
    com_detailed.to_csv(r'./com_detailed.csv', index=False)

    print('Successfully crawled {} jobs.'.format(job_list.shape[0]))


if __name__ == '__main__':
    # 想爬什么就改这里和存储位置，是实习僧的手机端搜索关键词，一页 20 个
    sxs_crawl(pages=50, kw='数据', c='北京')

Successfully crawled {} jobs list
Successfully crawled 783 jobs.


In [2]:
# 读 csv 文件
job_list   = pd.DataFrame(pd.read_csv(r'./job_list.csv'))
job_detail = pd.DataFrame(pd.read_csv(r'./job_detailed.csv'))

In [89]:
pd.set_option('display.max_columns', None)
job_detail.head()

,address,attraction,chance,city,cname,collected,cuuid,day,degree,delivered,endtime,ftype,iname,industry,info,invited,is_hr,job_label,job_other_info,job_requirements,logo,maxsal,mina_url,minsal,month,overdue,refresh,reslan,scale,url
0,北京,"['高速发展', '快速成长', '免费三餐']",面议,北京,作业帮,0,com_wp9oun0l9aur,4,本科,0,2019-06-05,1,实习生—社群运营J11946,计算机/互联网,<p><br>1、负责社群每日朋友圈及个人号的维护及运营；\r\r\n2、跟进部门物流状态并...,0,False,"{'is_ji': False, 'is_quick': False, 'is_summer...",NaN,NaN,https://sxsimg.xiaoyuanzhao.com/66/85/66079AF4...,200,pages/zwDetail/zwDetail?uuid=inn_84zlk9e1z6dg,150,3,0,2019-05-23 10:26:27,中文,500-2000人,https://www.shixiseng.com/intern/inn_84zlk9e1z6dg
1,北京朝阳区建国路81号 华贸中心1号写字楼25层,['发展空间大'],面议,北京,康泰纳仕,0,com_agi7umnontus,4,不限,0,2019-06-28,1,《悦游Condé Nast Traveler》编辑部实习生,文化传媒,<p>\r\n\t悦游Condé Nast Traveler》是一本将休闲与优雅、随兴与高端...,0,False,"{'is_ji': False, 'is_quick': False, 'is_summer...",NaN,NaN,https://sxsimg.xiaoyuanzhao.com/74/C0/7469733E...,150,pages/zwDetail/zwDetail?uuid=inn_x21ckrndrw28,100,6,0,2019-05-23 10:23:57,中文,150-500人,https://www.shixiseng.com/intern/inn_x21ckrndrw28
2,北京,"['高速发展', '快速成长', '免费三餐']",面议,北京,作业帮,0,com_wp9oun0l9aur,4,大专,0,2019-05-27,1,社群运营实习生J11919,计算机/互联网,<p>【岗位职责】\r\r\n1、负责Q群的管理工作，向用户介绍课程产品，促成学员购买\r\...,0,False,"{'is_ji': False, 'is_quick': False, 'is_summer...",NaN,NaN,https://sxsimg.xiaoyuanzhao.com/66/85/66079AF4...,200,pages/zwDetail/zwDetail?uuid=inn_rbavf46jzrje,150,3,0,2019-05-23 10:23:20,中文,500-2000人,https://www.shixiseng.com/intern/inn_rbavf46jzrje
3,北京市朝阳区朝外大街22号泛利大厦,['大牛团队'],可转正,北京,明亚,0,com_7uo8gxqsrel4,3,本科,0,2019-06-03,1,内容编辑实习生,金融,<p><strong>What you do：</strong> </p><p>1、<spa...,0,False,"{'is_ji': False, 'is_quick': False, 'is_summer...",NaN,NaN,https://sxsimg.xiaoyuanzhao.com/83/21/83CDEBCC...,150,pages/zwDetail/zwDetail?uuid=inn_25nrvfwnyqoe,100,4,0,2019-05-23 10:22:12,中文,2000人以上,https://www.shixiseng.com/intern/inn_25nrvfwnyqoe
4,北京市朝阳区朝外大街22号泛利大厦,['大牛云集'],可转正,北京,明亚,0,com_7uo8gxqsrel4,3,本科,0,2019-06-05,1,用户运营实习生,金融,<p>What you do：</p><p>1、系统化从事“随身保典”用户运营工作；</p>...,0,False,"{'is_ji': False, 'is_quick': False, 'is_summer...",NaN,NaN,https://sxsimg.xiaoyuanzhao.com/83/21/83CDEBCC...,150,pages/zwDetail/zwDetail?uuid=inn_tjkgzintcxjt,100,4,0,2019-05-23 10:22:08,中文,2000人以上,https://www.shixiseng.com/intern/inn_tjkgzintcxjt


In [90]:
job_list.head()

,ad_type,city,cname,day,deliver,ftype,industry,invite,is_hr,is_view,job_label,maxsal,minsal,month_num,name,refresh,scale,type,url,uuid
0,f,北京,作业帮,4,False,1,计算机/互联网,False,False,False,"{'is_ji': False, 'is_quick': False}",200,150,3,实习生—社群运营J11946,05-23,500-2000人,intern,https://sxsimg.xiaoyuanzhao.com/66/85/66079AF4...,inn_84zlk9e1z6dg
1,f,北京,康泰纳仕,4,False,1,文化传媒,False,False,False,"{'is_ji': False, 'is_quick': False}",150,100,6,《悦游Condé Nast Traveler》编辑部实习生,05-23,150-500人,intern,https://sxsimg.xiaoyuanzhao.com/74/C0/7469733E...,inn_x21ckrndrw28
2,f,北京,作业帮,4,False,1,计算机/互联网,False,False,False,"{'is_ji': False, 'is_quick': False}",200,150,3,社群运营实习生J11919,05-23,500-2000人,intern,https://sxsimg.xiaoyuanzhao.com/66/85/66079AF4...,inn_rbavf46jzrje
3,f,北京,明亚,3,False,1,金融,False,False,False,"{'is_ji': False, 'is_quick': False}",150,100,4,内容编辑实习生,05-23,2000人以上,intern,https://sxsimg.xiaoyuanzhao.com/83/21/83CDEBCC...,inn_25nrvfwnyqoe
4,f,北京,明亚,3,False,1,金融,False,False,False,"{'is_ji': False, 'is_quick': False}",150,100,4,用户运营实习生,05-23,2000人以上,intern,https://sxsimg.xiaoyuanzhao.com/83/21/83CDEBCC...,inn_tjkgzintcxjt


In [73]:
# 将数据库需要的内容存成字典，左边是数据库的字段名，右边必须是一维列表
data = {
    'id'    : list(range(1740,2740)),                                                               # 主键的位置，每次导入都需更新
    'source': [2]*1000,                                                                             # 参考数据库表格，实习僧为 2
    'source_position_id':[i for item in (job_list[['uuid']].values.tolist()) for i in item],        # 取出 DataFrame 的一列，转成列表
    'company_name': [i for item in (job_detail[['cname']].values.tolist()) for i in item],          # 但这个列表是二维列表，所以需要用
    'company_logo': [i for item in (job_detail[['logo']].values.tolist()) for i in item],           # 循环转成一维的
    'position_name': [i for item in (job_list[['name']].values.tolist()) for i in item],
    'city':[i for item in (job_list[['city']].values.tolist()) for i in item],
    'education':[i for item in (job_detail[['degree']].values.tolist()) for i in item],
    'salary': [i for item in (((np.array(job_list[['maxsal']])+np.array(job_list[['minsal']]))/2).tolist()) for i in item],
    'exceed': ['超过98%应届毕业生']*1000,
    'experience_requirements':['']*1000,
    'position_requirements':[i for item in (job_detail[['info']].values.tolist()) for i in item],
    'fringe_benefits':[i for item in (job_detail[['attraction']].values.tolist()) for i in item],
    'type':[2]*1000,
    'hot':[0]*1000,
    'create_time':[i for item in (job_detail[['refresh']].values.tolist()) for i in item],
    'poster':['ZJF']*1000
}

In [87]:
# 字典存成 DataFrame 再存成 csv 文件
job = pd.DataFrame(data)
job.to_csv("yunying1000.csv")

In [82]:
# 连数据库，传 DataFrame
engine = create_engine("mysql+pymysql://root:@localhost:3306/job?charset=utf8mb4")
job.to_sql(name = 'xcx_high_salary_jobs',con = engine,if_exists = 'append',index = False,index_label = False)

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '1740' for key 'PRIMARY'") [SQL: 'INSERT INTO xcx_high_salary_jobs (id, source, source_position_id, company_name, company_logo, position_name, city, education, salary, exceed, experience_requirements, position_requirements, fringe_benefits, type, hot, create_time, poster) VALUES (%(id)s, %(source)s, %(source_position_id)s, %(company_name)s, %(company_logo)s, %(position_name)s, %(city)s, %(education)s, %(salary)s, %(exceed)s, %(experience_requirements)s, %(position_requirements)s, %(fringe_benefits)s, %(type)s, %(hot)s, %(create_time)s, %(poster)s)'] [parameters: ({'id': 1740, 'source': 2, 'source_position_id': 'inn_84zlk9e1z6dg', 'company_name': '作业帮', 'company_logo': 'https://sxsimg.xiaoyuanzhao.com/66/85/66079AF4E39360D512F8664389B35385.png', 'position_name': '实习生—社群运营J11946', 'city': '北京', 'education': '本科', 'salary': 175.0, 'exceed': '超过98%应届毕业生', 'experience_requirements': '', 'position_requirements': '<p><br>1、负

def connect_wxremit_db():
    return pymysql.connect(host='localhost',
                           port=3306,
                           user='root',
                           password='',
                           database='job',
                           charset='utf8mb4')

def query_country_name(cc2):
    sql_str = ("SELECT id,source_position_id,position_name"
                + " FROM xcx_high_salary_jobs"
                + " WHERE id>%d" % (cc2))
    # logging.info(sql_str)

    con = connect_wxremit_db()
    cur = con.cursor()
    cur.execute(sql_str)
    rows = cur.fetchall()
    cur.close()
    con.close()

    # assert len(rows) == 1, 'Fatal error: country_code does not exists!'
    return rows[0][0]

In [77]:
query_country_name(1000)

1001